In [1]:
import pandas as pd 
import sklearn 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics 
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [3]:
data=pd.read_csv('Train.csv')
data.head()
data['Client Retention Flag']= data['Client Retention Flag'].map({'Yes': 1, 'No': 0})

In [9]:
data=data.drop(columns='Flag 5')

In [13]:
data=data.drop(columns=['Activity 5 Time Period  11','Activity 5 Time Period  10','Activity 5 Time Period  9','Activity 5 Time Period  8','Activity 5 Time Period  7','Activity 5 Time Period  6','Activity 5 Time Period  5','Activity 5 Time Period  4','Activity 5 Time Period  3','Activity 5 Time Period  2','Activity 5 Time Period  1','Activity 5 Time Period  0'
,'Activity 6 Time Period  11','Activity 6 Time Period  10','Activity 6 Time Period  9','Activity 6 Time Period  8','Activity 6 Time Period  7','Activity 6 Time Period  6','Activity 6 Time Period  5','Activity 6 Time Period  4','Activity 6 Time Period  3','Activity 6 Time Period  2','Activity 6 Time Period  1','Activity 6 Time Period  0'
,'Activity 7 Time Period  11','Activity 7 Time Period  10','Activity 7 Time Period  9','Activity 7 Time Period  8','Activity 7 Time Period  7','Activity 7 Time Period  6','Activity 7 Time Period  5','Activity 7 Time Period  4','Activity 7 Time Period  3','Activity 7 Time Period  2','Activity 7 Time Period  1','Activity 7 Time Period  0'
,'Activity 8 Time Period 11','Activity 8 Time Period 10','Activity 8 Time Period 9','Activity 8 Time Period 8','Activity 8 Time Period 7','Activity 8 Time Period 6','Activity 8 Time Period 5','Activity 8 Time Period 4','Activity 8 Time Period 3','Activity 8 Time Period 2','Activity 8 Time Period 1','Activity 8 Time Period 0'
                       ])

In [17]:
data.head()

,Client ID,Company ID,Client Contract Starting Month,Flag 1,Flag 2,Flag 3,Flag 4,Flag 6,Activity 1 Time Period 11,Activity 1 Time Period 10,...,Activity 4 Time Period 8,Activity 4 Time Period 7,Activity 4 Time Period 6,Activity 4 Time Period 5,Activity 4 Time Period 4,Activity 4 Time Period 3,Activity 4 Time Period 2,Activity 4 Time Period 1,Activity 4 Time Period 0,Client Retention Flag
0,10004519380,6294937,Jun,Multi Year,2,Yes,Unknown,Manufacturing,24,16,...,1,0,0,0,2,0,0,0,1,1
1,10004521476,6293796,Mar,Multi Year,7,No,Yes,Government,9,2,...,0,0,0,0,0,2,0,0,0,0
2,10004515403,6293911,Dec,Multi Year,2,No,Unknown,Technology and Telecom,1,11,...,0,0,0,1,0,0,1,0,1,1
3,10004511478,6293048,Feb,Multi Year,2,No,Unknown,Technology and Telecom,1,0,...,1,0,0,0,1,0,0,0,0,0
4,10004516858,6294364,Jun,Multi Year,7,No,Unknown,"Banking, Finance & Ins",16,14,...,0,1,0,1,1,1,0,0,0,1


In [19]:
data_encoded=pd.get_dummies(data)
data_encoded=data_encoded.drop(columns='Client ID')
pd.get_dummies(data_encoded,drop_first=True)
data_encoded=pd.get_dummies(data_encoded,columns=['Flag 2'])
#data_encoded.head()

In [20]:
Y=data_encoded['Client Retention Flag']
X=data_encoded.drop(columns='Client Retention Flag')

In [21]:
X.head()

,Company ID,Activity 1 Time Period 11,Activity 1 Time Period 10,Activity 1 Time Period 9,Activity 1 Time Period 8,Activity 1 Time Period 7,Activity 1 Time Period 6,Activity 1 Time Period 5,Activity 1 Time Period 4,Activity 1 Time Period 3,...,Flag 2_1,Flag 2_2,Flag 2_3,Flag 2_4,Flag 2_5,Flag 2_6,Flag 2_7,Flag 2_9,Flag 2_10,Flag 2_13
0,6294937,24,16,20,52,0,14,94,44,16,...,0,1,0,0,0,0,0,0,0,0
1,6293796,9,2,0,18,27,0,26,2,6,...,0,0,0,0,0,0,1,0,0,0
2,6293911,1,11,12,0,11,13,5,6,11,...,0,1,0,0,0,0,0,0,0,0
3,6293048,1,0,0,0,2,0,10,1,0,...,0,1,0,0,0,0,0,0,0,0
4,6294364,16,14,48,6,6,0,8,18,12,...,0,0,0,0,0,0,1,0,0,0


In [76]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 1000),
              'learning_rate': stats.uniform(0.01, 0.6),
              'subsample': stats.uniform(0.1, 0.9),
              'max_depth': [3, 4, 5, 6, 7, 8, 9,10,11],
              'colsample_bytree': stats.uniform(0.5, 0.9),
              'min_child_weight': [1, 2, 3, 4]
             }

numFolds = 5
kfold_5 = KFold(shuffle = True, n_splits = numFolds)

clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = kfold_5,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'roc_auc', 
                         error_score = 0, 
                         verbose = 3, 
                         n_jobs = -1)

In [77]:
train_X, test_X, train_Y, test_Y = train_test_split(X,Y,random_state=2)

In [78]:
clf.fit(train_X,train_Y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  25 | elapsed:   43.6s remaining:   16.9s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.3min finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
                   error_score=0,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logisti...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001633FF797F0>,
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10,
                                  

In [63]:
clf.cv_results_

{'mean_fit_time': array([ 0.11357436,  0.10477409,  0.10556793,  0.12476668, 22.92696505]),
 'std_fit_time': array([0.00823434, 0.0039166 , 0.01254868, 0.02350507, 3.40807984]),
 'mean_score_time': array([0.        , 0.        , 0.        , 0.        , 0.06163998]),
 'std_score_time': array([0.        , 0.        , 0.        , 0.        , 0.00922325]),
 'param_colsample_bytree': masked_array(data=[1.3189614716754545, 1.3060050445526443,
                    1.112599707629386, 1.0384992994987052,
                    0.9549114151605211],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.3129148494021973, 0.05034846839123267,
                    0.5534488144990509, 0.08442254360592294,
                    0.6081763390464386],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[4, 5, 5, 9, 4],
  

In [64]:
clf.cv_results_['mean_test_score']
clf.best_params_

{'colsample_bytree': 0.9549114151605211,
 'learning_rate': 0.6081763390464386,
 'max_depth': 4,
 'min_child_weight': 2,
 'n_estimators': 397,
 'subsample': 0.4262455970999188}

In [65]:
d_train = xgb.DMatrix(train_X, label=train_Y)
d_test = xgb.DMatrix(test_X, label=test_Y)
params = {
'colsample_bytree': 0.5432477190709075,
   'learning_rate': 0.33175415334759234,
   'max_depth': 9,
   'min_child_weight': 2,
   'n_estimators': 985,
   'subsample': 0.9529567150275902}
model2 = xgb.train(params, d_train, 2000, evals = [(d_test, "test")], verbose_eval=100, early_stopping_rounds=20)


[0]	test-rmse:0.458899
Will train until test-rmse hasn't improved in 20 rounds.


F:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Stopping. Best iteration:
[73]	test-rmse:0.346026



In [66]:
xgb_fea_imp=pd.DataFrame(list(model2.get_fscore().items()),
columns=['feature','importance']).sort_values('importance', ascending=False)
print('',xgb_fea_imp)

                                feature  importance
3                           Company ID        1827
0             Activity 1 Time Period 0         857
36            Activity 1 Time Period 1         827
1             Activity 1 Time Period 4         812
56            Activity 1 Time Period 2         795
24           Activity 1 Time Period 10         786
8             Activity 1 Time Period 6         773
7            Activity 1 Time Period 11         755
9             Activity 1 Time Period 9         728
37            Activity 1 Time Period 3         692
38            Activity 1 Time Period 8         670
43            Activity 1 Time Period 7         662
4             Activity 1 Time Period 5         646
5             Activity 2 Time Period 2         224
71            Activity 4 Time Period 4         209
28            Activity 2 Time Period 1         200
20            Activity 2 Time Period 4         197
41            Activity 4 Time Period 0         194
17            Activity 2 Time 

In [69]:
y_pred=model2.predict(d_test)
y_pred
for i in range(y_pred.size):
    y_pred[i]=round(y_pred[i])

In [70]:
metrics.accuracy_score(test_Y,y_pred)

0.8403284671532847

In [71]:
best_xg=clf.best_estimator_

In [72]:
y_pred=best_xg.predict(test_X)
y_pred
for i in range(y_pred.size):
    y_pred[i]=round(y_pred[i])

In [73]:
metrics.accuracy_score(test_Y,y_pred)

0.8298357664233577